In [0]:
%pip install -e ..

In [0]:
%pip install git+https://github.com/end-to-end-mlops-databricks-3/marvelous

In [0]:
%restart_python

In [0]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent / "src"))

In [0]:
# Databricks notebook source

import mlflow
from pyspark.sql import SparkSession

from hotel_reservations.config import ProjectConfig, Tags
from hotel_reservations.models.custom_model import CustomModel

In [0]:
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")

In [0]:
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
spark = SparkSession.builder.getOrCreate()
tags = Tags(**{"git_sha": "abcd12345", "branch": "week2"})

In [0]:
custom_model = CustomModel(config=config, tags=tags, spark=spark, code_paths=["../src"])

In [0]:
custom_model.load_data()

In [0]:
# Runs everything including feature preparation and model training
custom_model.log_model()

In [0]:
run_id = mlflow.search_runs(experiment_names=["/Shared/hotel-reservations-custom"]).run_id[0]

In [0]:
model = mlflow.pyfunc.load_model(f"runs:/{run_id}/pyfunc-lightgbm-hotel-model")

In [0]:
# Retrieve dataset for the current run
custom_model.retrieve_current_run_dataset()

In [0]:
# Retrieve metadata for the current run
custom_model.retrieve_current_run_metadata()

In [0]:
# Register model
custom_model.register_model()

In [0]:
# Predict on the test set

test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").limit(10)

X_test = test_set.drop(config.target).toPandas()
predictions_df = custom_model.load_latest_model_and_predict(X_test)